In [1]:
from google.colab import files

uploaded = files.upload()  # Upload your multimodal_only_samples.zip

Saving multimodal_only_samples.zip to multimodal_only_samples.zip


In [3]:
import zipfile
import os

zip_path = "multimodal_only_samples.zip"  # Replace with your actual file name if different
extract_path = "/content/fakeddit_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Unzipped files:")
os.listdir(extract_path)

Unzipped files:


['multimodal_only_samples']

In [4]:
import pandas as pd

# Update path to the correct directory
train_df = pd.read_csv('/content/fakeddit_data/multimodal_only_samples/multimodal_train.tsv', sep='\t')
val_df = pd.read_csv('/content/fakeddit_data/multimodal_only_samples/multimodal_validate.tsv', sep='\t')
test_df = pd.read_csv('/content/fakeddit_data/multimodal_only_samples/multimodal_test_public.tsv', sep='\t')

In [5]:
columns = ["clean_title", "image_url", "2_way_label"]
train_df = train_df[columns].dropna()
val_df   = val_df[columns].dropna()
test_df  = test_df[columns].dropna()

In [6]:
for df in [train_df, val_df, test_df]:
    df.rename(columns={"clean_title": "text", "image_url": "image_url", "2_way_label": "label"}, inplace=True)

In [7]:
train_df.head()

,text,image_url,label
0,my walgreens offbrand mucinex was engraved wit...,https://external-preview.redd.it/WylDbZrnbvZdB...,1
1,this concerned sink with a tiny hat,https://preview.redd.it/wsfx0gp0f5h11.jpg?widt...,0
2,hackers leak emails from uae ambassador to us,https://external-preview.redd.it/6fNhdbc6K1vFA...,1
3,puppy taking in the view,https://external-preview.redd.it/HLtVNhTR6wtYt...,1
4,i found a face in my sheet music too,https://preview.redd.it/ri7ut2wn8kv01.jpg?widt...,0


In [8]:
# Print original dataset sizes
print("Original dataset sizes:")
print(f"Training samples:   {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Test samples:       {len(test_df)}")

Original dataset sizes:
Training samples:   562466
Validation samples: 59169
Test samples:       59163


In [9]:
train_df = train_df.sample(n=5000, random_state=42)
val_df = val_df.sample(n=1000, random_state=42)
test_df = test_df.sample(n=1000, random_state=42)

In [10]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-6ehxi9og
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-6ehxi9og
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [12]:
from torch.utils.data import Dataset
from PIL import Image
import requests
from io import BytesIO
import clip
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")  # or another model

class FakedditURLDataset(Dataset):
    def __init__(self, dataframe, preprocess):
        self.df = dataframe
        self.preprocess = preprocess

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        url = row["image_url"]
        text = row["text"]
        label = int(row["label"])

        # Load image from URL
        try:
            response = requests.get(url, timeout=5)
            image = Image.open(BytesIO(response.content)).convert("RGB")
            image = self.preprocess(image)
        except:
            # Use a blank image if load fails
            image = Image.new("RGB", (224, 224), (255, 255, 255))
            image = self.preprocess(image)

        try:
            if len(text.split()) > 75:
                summary = summarizer(text, max_length=60, min_length=30, do_sample=False)[0]["summary_text"]
            else:
                summary = text
        except Exception as e:
            print(f"Summarization failed for sample {idx}: {e}")
            summary = text[:300]  # fallback

        tokenized_text = clip.tokenize([summary], truncate=True)[0]

        return image, tokenized_text, torch.tensor(label)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [13]:
from torch.utils.data import DataLoader
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)

train_dataset = FakedditURLDataset(train_df, preprocess)
val_dataset   = FakedditURLDataset(val_df, preprocess)
test_dataset  = FakedditURLDataset(test_df, preprocess)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=128)
test_loader  = DataLoader(test_dataset, batch_size=128)

100%|███████████████████████████████████████| 338M/338M [00:11<00:00, 31.8MiB/s]


In [14]:
import torch.nn as nn
from sklearn.metrics import accuracy_score

In [15]:
class FakeNewsClassifier(nn.Module):
    def __init__(self, embed_dim=512):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(embed_dim * 2, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 2)  # Binary output
        )

    def forward(self, image_feat, text_feat):
        x = torch.cat((image_feat, text_feat), dim=1)
        return self.fc(x)

In [16]:
print(f"Number of batches in train_loader: {len(train_loader)}")
print(f"Number of batches in val_loader: {len(val_loader)}")

Number of batches in train_loader: 40
Number of batches in val_loader: 8


In [17]:
!pip install tqdm

In [18]:
from tqdm import tqdm

print("Starting training...")

model = FakeNewsClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)

print(f"Number of batches in train_loader: {len(train_loader)}")

# Training loop
for epoch in range(5):  # Change epochs as needed
    print(f"\n=== Epoch {epoch+1} ===")

    model.train()
    total_loss = 0
    correct = 0
    total = 0

    # tqdm progress bar wrapper
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}")

    for batch_idx, (images, texts, labels) in progress_bar:
        print(f"\nBatch {batch_idx+1}")

        # Check batch contents
        print("Raw batch shapes:")
        print("Images:", images.shape)
        print("Texts:", texts.shape)
        print("Labels:", labels.shape)

        images, texts, labels = images.to(device), texts.to(device), labels.to(device)
        print("Moved batch to device")

        # Extract features using CLIP
        with torch.no_grad():
            print("Encoding image features...")
            image_features = clip_model.encode_image(images).float()
            print("Encoding text features...")
            text_features = clip_model.encode_text(texts).float()
            print("Image features:", image_features.shape)
            print("Text features:", text_features.shape)

        # Forward pass
        outputs = model(image_features, text_features)

        # Check output shapes for debugging
        print(f"Outputs shape: {outputs.shape}, Labels shape: {labels.shape}")

        # Compute loss
        loss = criterion(outputs, labels)
        print(f"Loss: {loss.item():.4f}")

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        # Accuracy calculation
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        accuracy = 100 * correct / total

        # Update progress bar
        progress_bar.set_postfix(loss=loss.item())
        avg_loss = total_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        print(f"Epoch {epoch+1} Complete - Avg Loss = {avg_loss:.4f} - Accuracy = {epoch_accuracy:.2f}%")

Starting training...
Number of batches in train_loader: 40

=== Epoch 1 ===


Epoch 1:   0%|          | 0/40 [00:00<?, ?it/s]


Batch 1
Raw batch shapes:
Images: torch.Size([128, 3, 224, 224])
Texts: torch.Size([128, 77])
Labels: torch.Size([128])
Moved batch to device
Encoding image features...
Encoding text features...


Epoch 1:   2%|▎         | 1/40 [00:40<26:18, 40.47s/it, loss=0.698]

Image features: torch.Size([128, 512])
Text features: torch.Size([128, 512])
Outputs shape: torch.Size([128, 2]), Labels shape: torch.Size([128])
Loss: 0.6982
Epoch 1 Complete - Avg Loss = 0.0175 - Accuracy = 43.75%

Batch 2
Raw batch shapes:
Images: torch.Size([128, 3, 224, 224])
Texts: torch.Size([128, 77])
Labels: torch.Size([128])
Moved batch to device
Encoding image features...
Encoding text features...


...

Image features: torch.Size([128, 512])
Text features: torch.Size([128, 512])
Outputs shape: torch.Size([128, 2]), Labels shape: torch.Size([128])
Loss: 0.1468
Epoch 5 Complete - Avg Loss = 0.1870 - Accuracy = 92.37%

Batch 39
Raw batch shapes:
Images: torch.Size([128, 3, 224, 224])
Texts: torch.Size([128, 77])
Labels: torch.Size([128])
Moved batch to device
Encoding image features...
Encoding text features...


Epoch 5:  98%|█████████▊| 39/40 [22:07<00:33, 33.29s/it, loss=0.172]

Image features: torch.Size([128, 512])
Text features: torch.Size([128, 512])
Outputs shape: torch.Size([128, 2]), Labels shape: torch.Size([128])
Loss: 0.1716
Epoch 5 Complete - Avg Loss = 0.1913 - Accuracy = 92.39%

Batch 40
Raw batch shapes:
Images: torch.Size([8, 3, 224, 224])
Texts: torch.Size([8, 77])
Labels: torch.Size([8])
Moved batch to device
Encoding image features...
Encoding text features...


Epoch 5: 100%|██████████| 40/40 [22:09<00:00, 33.23s/it, loss=0.368]

Image features: torch.Size([8, 512])
Text features: torch.Size([8, 512])
Outputs shape: torch.Size([8, 2]), Labels shape: torch.Size([8])
Loss: 0.3682
Epoch 5 Complete - Avg Loss = 0.2005 - Accuracy = 92.38%


In [23]:
from sklearn.metrics import classification_report
def evaluate_model(model, data_loader, clip_model, device):
    model.eval()
    all_preds = []
    all_labels = []
    total_loss = 0

    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for images, texts, labels in tqdm(data_loader, desc="Evaluating"):
            images = images.to(device)
            texts = texts.to(device)
            labels = labels.to(device)

            # Extract CLIP features
            image_features = clip_model.encode_image(images).float()
            text_features = clip_model.encode_text(texts).float()

            # Forward pass through classifier
            outputs = model(image_features, text_features)

            # Compute loss
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # Get predictions
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute accuracy
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"\n✅ Evaluation Complete — Accuracy: {accuracy * 100:.2f}%")
    print(f"📉 Average Loss: {total_loss / len(data_loader):.4f}")
    print("\n📊 Classification Report:\n", classification_report(all_labels, all_preds, target_names=["Real", "Fake"]))

    return accuracy

In [24]:
print("\n=== Evaluating on Validation Set ===")
evaluate_model(model, val_loader, clip_model, device)

print("\n=== Evaluating on Test Set ===")
evaluate_model(model, test_loader, clip_model, device)


=== Evaluating on Validation Set ===


Evaluating: 100%|██████████| 8/8 [04:21<00:00, 32.65s/it]



✅ Evaluation Complete — Accuracy: 88.60%
📉 Average Loss: 0.2715

📊 Classification Report:
               precision    recall  f1-score   support

        Real       0.92      0.88      0.90       579
        Fake       0.84      0.90      0.87       421

    accuracy                           0.89      1000
   macro avg       0.88      0.89      0.88      1000
weighted avg       0.89      0.89      0.89      1000


=== Evaluating on Test Set ===


Evaluating: 100%|██████████| 8/8 [04:32<00:00, 34.03s/it]


✅ Evaluation Complete — Accuracy: 88.60%
📉 Average Loss: 0.2801

📊 Classification Report:
               precision    recall  f1-score   support

        Real       0.95      0.86      0.90       608
        Fake       0.81      0.93      0.86       392

    accuracy                           0.89      1000
   macro avg       0.88      0.89      0.88      1000
weighted avg       0.89      0.89      0.89      1000



0.886

In [29]:
torch.save(model.state_dict(), "clip_fakenews_classifier.pth")
print(" Model saved.")

 Model saved.


In [26]:
from google.colab import files
files.download("clip_fakenews_classifier.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
model = FakeNewsClassifier().to(device)
model.load_state_dict(torch.load("clip_fakenews_classifier.pth"))
model.eval()

FakeNewsClassifier(
  (fc): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=512, out_features=128, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=128, out_features=2, bias=True)
  )
)

In [28]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 125.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.0 MB/s eta 0:00:00


In [30]:
import gradio as gr

def predict_from_gradio(text, image):
    model.eval()

    try:
        image_input = preprocess(image).unsqueeze(0).to(device)
    except:
        image_input = preprocess(Image.new("RGB", (224, 224), (255, 255, 255))).unsqueeze(0).to(device)

    text_input = clip.tokenize([text]).to(device)

    with torch.no_grad():
        image_features = clip_model.encode_image(image_input).float()
        text_features = clip_model.encode_text(text_input).float()
        outputs = model(image_features, text_features)
        _, predicted = torch.max(outputs, 1)

    label = predicted.item()
    return "Real News" if label == 0 else "Fake News"

In [33]:
interface = gr.Interface(
    fn=predict_from_gradio,
    inputs=[
        gr.Textbox(label="News Headline / Description", placeholder="Enter the news content here..."),
        gr.Image(type="pil", label="Associated Image", sources=["upload", "clipboard"])
    ],
    outputs=gr.Textbox(label="Result"),
    title="📰 Fake News Detection",
    description="This app uses OpenAI's CLIP model to detect fake news based on text + image."
)

interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2e816617e35b9c9ccb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
